# oneAPI

Un camino hacia una computación acelerada libre de los inconvenientes económicos y técnicos de los modelos de programación propietarios.

- Programación de arquitectura cruzada que provee libertad para elegir el mejor hardware
- Basado en estándares industriales y especificaciones abiertas
- Expone características de rendimiento de vanguardia de hardware reciente
- Compatible con lenguajes y modelos de programación existentes para alto rendimiento 

## Introducción a SYCL

- SYCL es un lenguaje de programación basado en C++ y orientado para programación paralela

- Un programa SYCL tiene dos partes, **host** y **device**, que pueden en principio ser codificadas en el mismo archivo fuente (**single source**)

- A diferencia de C, SYCL requiere de dos sistemas de compilación, uno para el **host** y otro para el **device** deseado. Sin embargo, al compilar, solo un ejecutable es producido, y este ejecutable es usado para ambos host y device

- SYCL es de alto nivel de abstracción y puede ser aplicado a dispositivos de cómputo heterogéneos

### Implementaciones de SYCL

SYCL está disponible para muchos API backends y dispositivos electrónicos tal y como se presenta en la siguiente figura.

![alt_text](2020-05-sycl-landing-page-02b_1.jpg)
*Fuente: https://www.khronos.org/api/index_2017/sycl*

### Cómo implementar SYCL?

Para implementar un código SYCL se requiere los siguientes implementos:

1. SYCL API o la librería de C++ que contiene las aplicaciones de SYCL

2. SYCL runtime o tiempo de corrida que programa y ejecuta el código, lo compila, carga la data, los kernels, etc

3. Dispositivo electrónico (Host) que emula el backend de SYCL

4. Un backend como OpenCL

5. SYCL compilador que es similar a un compilador de C++

SYCL es una extensión de C++ y usa su misma sintaxis, sólo añadiendo comandos de paralelismo. En C, nosotros también usamos comandos de paralelización denominados "pragma" (ver clases del curso pasado). En si, tanto los comandos de paralalismo en SYCL como los pragmas en C son equivalentes en el propósito. La diferencia está en la flexibilidad y la responsabilidad que recae en el usuario. Los pragmas en C ofrecen paralización con poca intervención del usuario y sin realizar mayor cambio al código original. En SYCL, la implementación paralela es manual, y se puede tener mayor control del código, flexibilidad y potencialmente mayor rendimiento. Sin embargo dada su implementación manual, es más complejo de implementar.   

## El "Hello World" de SYCL

Un ejemplo del tipo "Hello world" en C no es un práctico para SYCL que es orientado a programación paralela por lo que vamos a trabajar con un ejemplo que pueda ser paralizable. Para ello vamos a usar el ejemplo dado por Intel en el siguiente repositorio (https://github.com/oneapi-src/oneAPI-samples/blob/master/DirectProgramming/C%2B%2BSYCL/Jupyter/oneapi-essentials-training/01_oneAPI_Intro/src/simple.cpp)

In [ ]:
%%writefile simple.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>
using namespace sycl;
static const int N = 16;
int main(){
  //# define queue which has default device associated for offload
  queue q;
  std::cout << "Device: " << q.get_device().get_info<info::device::name>() << "\n";

  //# Unified Shared Memory Allocation enables data access on host and device
  int *data = malloc_shared<int>(N, q);

  //# Initialization
  for(int i=0; i<N; i++) data[i] = i;

  //# Offload parallel computation to device
  q.parallel_for(range<1>(N), [=] (id<1> i){
    data[i] *= 2;
  }).wait();

  //# Print Output
  for(int i=0; i<N; i++) std::cout << data[i] << "\n";

  free(data, q);
  return 0;
}

Este archivo lo estamos creando desde Jupyter Notebooks, pero también podríamos crearlos desde Devcloud, usando las funciones "mkdir" para crear la carpeta de trabajo y "touch" para crear el archivo en blanco que lo podemos guardar como "simple.cpp". Nótese que la extensión del archivo es ".cpp" dado que sycl es una extensión de C++ (.cpp es extensión de C++).

### Elementos del Código

**1. *#include <sycl/sycl.hpp>*** 

Archivo de cabecera o librería estándar de SYCL con todas las definiciones necesarias para correr el programa. 

**2. using namespace sycl;**

Espacios de nombres o "namespace" (C++): permiten definir espacios, en este caso sycl tanto para CPU como para GPU (Tomar en cuenta que solo alguno de los elementos del código corren en GPU, no todo). Si en caso no se definiera al espacio de nombre sycl, para llamar a las funciones de este paquete tendriamos que escribir de la siguiente forma:

**sycl::queue q;** similar a **std::cout ...**

**3. static const int N = 16;** 

Estamos definiendo una variable del tipo integer "N" con un valor fijo de 16

**4. queue q;**

Dentro de "main()" que es como definimos empezamos el programa en C/C++, definimos un "queue q" (elemento propio de SYCL). Un "queue" entrega **command groups** para ser ejecutados por el “SYCL runtime”. No confundir con el *queue* usado para subir un job al cluster. "queue" o "q" que es como lo estamos definiendo a partir de ahora en el programa incluye muchas otras funciones como por ejemplo una función para obtener información del dispositivo dónde vamos a ejecutar el programa. Esta función es llamada *"get_device"* y es llamada en el programa a través de la notación punto "." que es propia de la programación orientada a objetos de C++. Cada objeto en C++ tiene características propias y funciones miembros y estás se llaman a través de la notación ".":

**q.get_device().get_info**

**5. std::cout << "Device: " << q.get_device().get_info<info::device::name>() << "\n";**

En esta línea vamos a imprimir información del dispositivo usado a través de las funciones de "q", usando notación punto "."

**6. int *data = malloc_shared<int>(N, q);**

Lugar de memoria con malloc_shared (memoria unificada) que tanto el CPU y GPU lo pueden ver. Es parte de la sintáxis de SYCL. Es una memoria para guardar enteros (tamaño N) y van a ser accedidos mediante el puntero "*data*". En C, el compilador asigna memoria y nosotros no intervenimos, pero con SYCL ese proceso es manual. Las variables punteros o pointer (*data*) que nos permite apuntar a direcciones de memoria RAM (espacios de memoria RAM), almacena memoria y se define con un "*". 

**7. for(int i=0; i<N; i++) data[i] = i;**

For loop. Aquí accedemos a los elementos de data como un arreglo y llenamos los valores de data con números de 0 a 15.

**8.   Las tres líneas siguientes son pura sintáxis de SYCL**

    q.parallel_for(range<1>(N), [=] (id<1> i){

    data[i] *= 2;

    }).wait();


- Estas tres líneas difieren de C++, y son las únicas 3 líneas que se ejecuten en el GPU. Las líneas anteriores, todas se ejecutan en el CPU. Este programa comienza ejecutándose en el CPU (host) en el tiempo 0 y el GPU está en espera de algún comando para empezar a ejecutar y este empieza en *q.parallel_for*. 
- *parallel_for* es una función dentro de queue (q) y tenemos un número N de threats y los vamos a identificar con la variable i.  
- El código que se ejecuta en el GPU es multiplicar cada elemento de la memoria apuntada por data por 2. Una alternativa equivalente de *= es *data[i] = data[i]*2;*
- wait indica que el GPU va a esperar a que todo termine para que luego vuelva a comenzar a ejecutar el CPU. Cada threat es independiente y no todos podrían terminar al mismo tiempo y por ello es recomendable usar una función tipo *"wait"*.

**9.   for(int i=0; i<N; i++) std::cout << data[i] << "\n";**

Imprimimos en contenido de data que son el doble de los valores originales.

**10.   free(data, q);**

Es una función de SYCL para liberar memoria. La función malloc ocupa la memoria RAM y cuando ya no la necesitamos podemos liberar la memoria con la función *free*. En C, usamos el std para asignar memoria y un vector por ejemplo y la libería std es la que se encarga de liberar la memoria no utilizada de manera automática (garbage collection). En SYCL, lo tenemos que hacer en forma manual. 

## Correr el código "simple.cpp"

1. Para poder correr este código, primero debemos conectarnos a DevCloud, así que el primer paso es abrir DevCloud a través de MobaXterm o del terminar usando el comando *ssh devcloud*
2. Luego vamos a conectarnos a un nodo interactivo tomando en cuenta que es no es una buena práctica trabajar en el nodo de inicio.

    *qsub -I -l nodes=1:gen11:ppn=2 -d .* (ejemplo)

    Hay que tomar en cuenta que estamos conectando a un nodo que tiene gpu que es el tema del curso. 
3. Luego vamos a usar el comando *sycl-ls* para verificar qué plataformas en nuestro nodo pueden utilizar (correr) formato sycl. Esto es recomendable realizar antes de ejecutar nuestro código en sycl. 

    ![alt text](sycl_ls.jpg)

En este nodo particular tenemos incluido FPGA (fichas de lego de los chip sin arquitectura definida dentro del chip), CPU y GPUs
4. Para ejecutar nuestro archivo *simple.cpp* primero debemos compilar nuestro archivo fuente y obtener nuestro archivo binario ejecutable tal y como lo hacemos con un archivo fuente en C. Para eso usamos el siguiente comando (no olvidar que primero debemos ir a la carpeta donde tenemos guardado nuestro proyecto):

    *dpcpp -o simple simple.cpp*

    En este archivo, el compilador es *dpcpp*, el nombre del archivo binario será *simple*, y el archivo fuente es *simple.cpp*

5. Luego ejecutamos el archivo binario que ha sido creado:

    *./simple*

    ![alt text](results.jpg)